In [ ]:
import requests
import pandas as pd
from datetime import datetime

In [ ]:
df = pd.read_csv('./AQI/Data/station_location.csv')
df.head()

,Station,lat,lon
0,"Naharlagun, Naharlagun, India",27.103358,93.679645
1,"Devaraj Urs Badavane, Davanagere, India",14.475800,75.905200
2,"Vinoba Nagara, Shivamogga, India",13.940000,75.555917
3,"Urban, Chamarajanagar, India",11.553580,76.555210
4,"Tamaka Ind. Area, Kolar, India",13.127629,78.157012


In [ ]:
# Total number of Stations
df.count()

Station    153
lat        153
lon        153
dtype: int64

In [ ]:
lat_list = df['lat'].tolist()
lon_list = df['lon'].tolist()
station = df['Station'].tolist()
coords_list = list(map(lambda x, y, z:(x,y,z), lat_list, lon_list, station))

#### Getting data using API

* Get the OpenWeathermap API KEY from [Here](https://openweathermap.org/api/air-pollution)

In [ ]:
hist_data_list = []
start_date = 1606501800   # 28-Nov-2020
end_date =  1644431400    # 10-Feb-2022
API_KEY = "{API_KEY}"
for item in coords_list:
    openweather_url = f"http://api.openweathermap.org/data/2.5/air_pollution/history?lat={item[0]}&lon={item[1]}&start={start_date}&end={end_date}&appid={API_KEY}"
    response = requests.get(openweather_url)
    data = response.json()
    for i in range(len(data['list'])):
        data_dict = {
            "Station": item[2],
            "Date":datetime.utcfromtimestamp(data.get('list')[i].get("dt")).strftime('%m-%d-%Y'),
            "Time":datetime.utcfromtimestamp(data.get('list')[i].get("dt")).strftime('%H:%M:%S'),
            "latitude": data.get('coord').get('lat'),
            "longitude": data.get('coord').get('lon'),
            "PM25":data.get('list')[i].get('components').get("pm2_5") if 'pm2_5' in data.get('list')[i].get('components') else "null",
            "PM10":data.get('list')[i].get('components').get("pm10") if 'pm10' in data.get('list')[i].get('components') else "null",
            "CO":data.get('list')[i].get('components').get("co") if 'co' in data.get('list')[i].get('components') else "null",
            "NO":data.get('list')[i].get('components').get("no") if 'no' in data.get('list')[i].get('components') else "null",
            "NO2":data.get('list')[i].get('components').get("no2") if 'no2' in data.get('list')[i].get('components') else "null",
            "NH3":data.get('list')[i].get('components').get("nh3") if 'nh3' in data.get('list')[i].get('components') else "null",
            "O3":data.get('list')[i].get('components').get("o3") if 'o3' in data.get('list')[i].get('components') else "null",
            "SO2":data.get('list')[i].get('components').get("so2") if 'so2' in data.get('list')[i].get('components') else "null",
            }
        hist_data_list.append(data_dict)
history_dataDF = pd.DataFrame(hist_data_list)

In [ ]:
history_dataDF.head()

,Station,Date,Time,latitude,longitude,PM25,PM10,CO,NO,NO2,NH3,O3,SO2
0,"Naharlagun, Naharlagun, India",11-27-2020,19:00:00,27.1034,93.6796,6.78,9.85,143.53,0.0,0.39,0.71,13.77,0.09
1,"Naharlagun, Naharlagun, India",11-27-2020,20:00:00,27.1034,93.6796,6.39,8.71,143.53,0.0,0.30,0.72,15.38,0.07
2,"Naharlagun, Naharlagun, India",11-27-2020,21:00:00,27.1034,93.6796,6.15,8.12,143.53,0.0,0.28,0.76,16.09,0.06
3,"Naharlagun, Naharlagun, India",11-27-2020,22:00:00,27.1034,93.6796,6.00,7.79,143.53,0.0,0.28,0.77,16.45,0.06
4,"Naharlagun, Naharlagun, India",11-27-2020,23:00:00,27.1034,93.6796,5.87,7.53,146.87,0.0,0.29,0.72,17.70,0.06


In [ ]:
history_dataDF.to_csv("./AQI/2020-2022_history_data.csv",index=False)